# Advanced Lane Finding Project
* The goals / steps of this project are the following:
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## First, I'll compute the camera calibration using chessboard images

In [3]:
import numpy as np                 # Matrix math
import cv2                         # OpenCV for image processing
import glob                        # Filename pattern matching
import matplotlib.pyplot as plt    # Data visualization libary
import matplotlib.image as mpimg   # Image processing library
# Visualizations will be shown in the notebook
%matplotlib inline                 

## Camera Calibration

In [5]:
def calibration_points(do_plot=False, do_file=False):
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
    objp = np.zeros((6*9, 3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    objpoints = []                     # 3D points in real world space
    imgpoints = []                     # 2D points in image plain

    # List of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    print('Num of calibration images: {0}'.format(len(images)))

    # Step through the list and search for chessboard corners
    for img_id, fname in enumerate(images): 
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Grayscale
        
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
        # If found - add object points, add image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9, 6), corners, ret)
            # Draw the plot
            if do_plot:
                plt.imshow(img)
                plt.show()
            # Save to the file
            if do_file:
                write_name = 'corners_' + str(img_id) + '.jpg' # Name to give image
                cv2.imwrite(write_name, img) # Write and save file locally
    return objpoints, imgpoints

## Distortion Correction

In [ ]:
import pickle

def pickle_dump(mtx, dist):
    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump(dist_pickle, open('wide_dist_pickle.p', 'wb'))
    
def pickle_load():
    # Getting back the camera calibration result:
    with open('wide_dist_pickle.p', 'rb') as f:
        dist_pickle = pickle.load(f)
        return dist_pickle['mtx'], dist_pickle['dist']
    
def calibrate_camera(img):
    img_size = (img.shape[1], img.shape[0])
    # Do camera calibration given object points and image points
    objpoints, imgpoints = calibration_points(do_plot=False, do_file=False)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    # Save the camera calibration result
    pickle_dump(mtx, dist)
    return mtx, dist

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # If line detected
        self.detected = False  
        # The x values of the last n fits
        self.recent_xfitted = collections.deque(12*[0.0, 0.0, 0.0], 12)
        # Mean x values of best fit line
        self.bestx = None
        # Mean polynomial coefficients 
        self.best_fit = None  
        # Polynomial coefficients for fit
        self.current_fit = [np.array([False])]  
        # Curvature radius of line in meters
        self.radius_of_curvature = None 
        # Meters from center of the line
        self.line_base_pos = None
        # Difference in old and new fit coefficients 
        self.diffs = np.array([0,0,0], dtype='float') 
        # The x values for detected line pixels
        self.allx = None
        # The y values for detected line pixels
        self.ally = None